In [ ]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Software_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42605767|R3EFW2STIYIY0I|B00MUTIDKI|     248732228|McAfee 2015 Inter...|        Software|          1|            2|          2|   N|                Y|I was very disapp...|I was very disapp...| 2015-08-31|
|         US|   51771800|R12NR0R5A9F7FT|B00EPACNUG|     531462352|Hallmark Card Stu...|        Software|          5|    

In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show()
vine_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3EFW2STIYIY0I|          1|            2|          2|   N|                Y|
|R12NR0R5A9F7FT|          5|            0|          0|   N|                Y|
|R1LSH74R9XAP59|          2|            0|          1|   N|                Y|
|R1QXUNTF76K7L6|          2|            0|          0|   N|                Y|
|R2F7DR75PS8NKT|          5|            0|          0|   N|                Y|
|R2C1DJSCC8UFS6|          3|            0|          0|   N|                Y|
|R1AXGS1W4YFXMX|          1|            0|          2|   N|                Y|
|R1XU1B93402SYJ|          1|            1|          1|   N|                Y|
|R2U432NB3OPVR0|          5|            0|          0|   N|                Y|
|R3R6FIMIOQ5SP9|          5|            0|          0|   N|     

341931

In [ ]:
vine_filtered_df = vine_df.filter(vine_df.total_votes >= 20)
vine_filtered_df.show()
vine_filtered_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R26CC1STPSUHBS|          1|           27|         39|   N|                N|
| R7K83D59T9KWS|          1|           16|         26|   N|                N|
|R2ATG5BPVILHUM|          1|           16|         25|   N|                N|
|R1Q71TAM5ZOYTK|          3|           57|         61|   N|                N|
| RR8HYI7C2XKMU|          5|           25|         25|   N|                Y|
|R1VDCSJE4CJJIR|          1|           88|         97|   N|                Y|
|R2PZMSDKOO1Q6W|          1|           29|         43|   N|                Y|
|R39OOYN0DOLINK|          3|           91|        111|   N|                N|
|R1PLNHJZ8BKIBG|          1|           44|         46|   N|                Y|
|R14G4YQXL17REH|          1|            4|         28|   N|     

20216

In [ ]:
vine_filtered2_df = vine_filtered_df.filter((vine_filtered_df.helpful_votes/vine_filtered_df.total_votes) >= 0.5)
vine_filtered2_df.show()
vine_filtered2_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R26CC1STPSUHBS|          1|           27|         39|   N|                N|
| R7K83D59T9KWS|          1|           16|         26|   N|                N|
|R2ATG5BPVILHUM|          1|           16|         25|   N|                N|
|R1Q71TAM5ZOYTK|          3|           57|         61|   N|                N|
| RR8HYI7C2XKMU|          5|           25|         25|   N|                Y|
|R1VDCSJE4CJJIR|          1|           88|         97|   N|                Y|
|R2PZMSDKOO1Q6W|          1|           29|         43|   N|                Y|
|R39OOYN0DOLINK|          3|           91|        111|   N|                N|
|R1PLNHJZ8BKIBG|          1|           44|         46|   N|                Y|
|R3UTVKJF44E0R6|          1|           46|         69|   N|     

17762

In [ ]:
#Get the Vine Paid Reviews
vine_paid_reviews_df = vine_filtered2_df.filter(vine_filtered2_df.vine == 'Y')
vine_paid_reviews_df.show(5)
print("Total number of Vine reviews:"+ str(vine_paid_reviews_df.count()))

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R28EIR3Z6RRUNN|          2|           50|         60|   Y|                N|
|R2YO2RESXZEWOF|          4|           24|         25|   Y|                N|
| R8GEE2EW3D6S9|          3|           49|         54|   Y|                N|
|R33W9V1MX6WKPS|          4|           47|         53|   Y|                N|
|R24E3W2ZRIFG8C|          5|           91|         93|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows

Total number of Vine reviews:248


In [ ]:
#Get the Vine Unpaid Reviews
vine_unpaid_reviews_df = vine_filtered2_df.filter(vine_filtered2_df.vine == 'N')
vine_unpaid_reviews_df.show(5)
print("Total number of non-Vine reviews:"+ str(vine_unpaid_reviews_df.count()))

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R26CC1STPSUHBS|          1|           27|         39|   N|                N|
| R7K83D59T9KWS|          1|           16|         26|   N|                N|
|R2ATG5BPVILHUM|          1|           16|         25|   N|                N|
|R1Q71TAM5ZOYTK|          3|           57|         61|   N|                N|
| RR8HYI7C2XKMU|          5|           25|         25|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows

Total number of non-Vine reviews:17514


In [ ]:
print("Total Number of Reviews: "+ str(vine_filtered2_df.count()))
print("Total Number of 5-Star Reviews: " + str(vine_filtered2_df.filter(vine_filtered2_df.star_rating == 5).count()))
print("Total Number of 5-Star Paid Reviews: " + str(vine_paid_reviews_df.filter(vine_paid_reviews_df.star_rating == 5).count()))
print("Total Number of 5-Star Unpaid Reviews: " + str(vine_unpaid_reviews_df.filter(vine_unpaid_reviews_df.star_rating == 5).count()))

Total Number of Reviews: 17762
Total Number of 5-Star Reviews: 5256
Total Number of 5-Star Paid Reviews: 102
Total Number of 5-Star Unpaid Reviews: 5154
